In [1]:
# Using test data to build/prototype-out a ML model
    # Reshaping, encoding, normalization
    # Input, layer, and output specification
    # Force keras/tensorflow to use GPU backend

In [1]:
# Imports
import pandas as pd
import numpy as np
import json
import datetime 
import matplotlib.pyplot as plt
%matplotlib inline

# Pymongo import for connection to local client DB
import pymongo
from pymongo import MongoClient

# ML Imports 
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, GRU
from keras.models import load_model
from keras import backend as K

# Preprocessing Imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from keras.utils import to_categorical 

# Import to check check for GPU availability for tensorflow backend
from tensorflow.python.client import device_lib

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
# Mongodb connection for live feed of data into model
    # Implementation after verification of test data -> model prototype input layer working

In [2]:
# Verify GPU availability for tensorflow backend
print(device_lib.list_local_devices())
print("==============================================")
print(K.tensorflow_backend._get_available_gpus())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7530173190370327337
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9222031934
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14343468339737875308
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
['/job:localhost/replica:0/task:0/device:GPU:0']


In [3]:
# Read in test data for L2 orderbook state (bids + asks)
    # Read in test data for subsquent L2 orderbook update states (L2 updates to bid + asks)
snapshot_asks_df = pd.read_csv("test_data/snapshot_asks.csv")
snapshot_bids_df = pd.read_csv("test_data/snapshot_bids.csv")
l2update_df = pd.read_csv("test_data/l2update.csv", dtype ={'changes':object})

In [4]:
print(snapshot_asks_df.head())
print(snapshot_bids_df.head())

     price       size
0  8337.67  35.733244
1  8337.70   1.000000
2  8337.71   1.198250
3  8339.77   0.048600
4  8339.80   0.098500
     price      size
0  8337.66  1.922593
1  8337.00  0.001000
2  8336.00  0.001000
3  8335.83  0.900000
4  8335.65  0.010000


In [5]:
l2update_df.head()

,_id,time,side,price,size
0,5ad3dc2ee3ae711d3ca939a7,2018-04-15T23:11:43.047Z,buy,8300.0,33.484726
1,5ad3dc2ee3ae711d3ca939a8,2018-04-15T23:11:43.060Z,buy,8300.0,33.484726
2,5ad3dc2ee3ae711d3ca939a9,2018-04-15T23:11:43.062Z,buy,8300.0,33.484726
3,5ad3dc2ee3ae711d3ca939aa,2018-04-15T23:11:43.065Z,buy,8300.0,33.484726
4,5ad3dc2ee3ae711d3ca939ac,2018-04-15T23:11:43.071Z,buy,8300.0,33.484726
